In [1]:
!pip install jieba
!pip install refo
!pip install rdflib

In [0]:
from refo import finditer, Predicate, Star, Any

import re
import rdflib
import jieba.posseg as pseg

Knowledge

In [0]:
URI_PREFIX = 'http://kgdemo.com/'

triples = [
  ('宝马', 'is_what', '宝马（BMW）是德国豪华汽车品牌'),
  ('宝马', 'is_how', '德系大品牌值得信赖，各方面口碑都很好'),
  ('宝马', 'is_compared', '各有千秋，但是人生苦短，我选宝马'),
]

graph = rdflib.Graph()
resources = set([r for triple in triples for r in triple])
resource2uri = {r: URI_PREFIX + r for r in resources}
uri2resource = {uri: r for r, uri in resource2uri.items()}
for (s, p, o) in triples:
  s_uri = rdflib.URIRef(resource2uri[s])
  p_uri = rdflib.URIRef(resource2uri[p])
  o_uri = rdflib.URIRef(resource2uri[o])
  graph.add((s_uri, p_uri, o_uri))

Rule

In [0]:
class W(Predicate):
  def __init__(self, token='.*', pos='.*'):
    self.token = re.compile(token + '$')
    self.pos = re.compile(pos + '$')
    super(W, self).__init__(self.match)
                  
  def match(self, word):
    m1 = self.token.match(word.token)
    m2 = self.pos.match(word.pos)
    return m1 and m2


class Rule(object):
  def __init__(self, condition=None, action=None):
    self.condition = condition
    self.action = action
          
  def apply(self, sentence):
    matches = []
    for m in finditer(self.condition, sentence):
      i, j = m.span()
      matches.extend(sentence[i:j])
    return self.action(matches)


class Word(object):
  def __init__(self, token, pos):
    self.token = token
    self.pos = pos

In [0]:
def what_is_xxx(matches):
  if len(matches) > 0:
    print("Query:", "  ".join([word.token+'|'+word.pos for word in matches]))
    print("Template: 'what is xxx?'")
    for word in matches:
      if word.pos == 'nr':
        return (
          """
          PREFIX : <%s>
          
          SELECT DISTINCT ?o WHERE {
              :%s :is_what ?o .
          }
          """ % (URI_PREFIX, word.token)
        )


def how_is_xxx(matches):
  if len(matches) > 0:
    print("Query:", "  ".join([word.token+'|'+word.pos for word in matches]))
    print("Template: 'how is xxx?'")
    for word in matches:
      if word.pos == 'nr':
        return (
          """
          PREFIX : <%s>
          
          SELECT DISTINCT ?o WHERE {
              :%s :is_how ?o .
          }
          """ % (URI_PREFIX, word.token)
        )


def xxx_compared_to(matches):
  if len(matches) > 0:
    print("Query:", "  ".join([word.token+'|'+word.pos for word in matches]))
    print("Template: 'xxx compared to xxx?'")
    for word in matches:
      if word.pos == 'nr' or word.pos == 'v':
        return (
          """
          PREFIX : <%s>
          
          SELECT DISTINCT ?o WHERE {
              :%s :is_compared ?o .
          }
          """ % (URI_PREFIX, word.token)
        )

In [0]:
rules = [                                                 
  Rule(condition = (W(pos='nr') | W(pos='v')) + W('和') + (
                    W(pos='nr') | W(pos='v')) + Star(W('比')) + (
                    W('怎么样') | W('怎样') | (W('哪个') + (W('好') | W('更好') ) )),
       action = xxx_compared_to),  

  Rule(condition = W(pos='nr') + Star(Any(), greedy=False) + \
                   (W('怎么样') | W('怎样') | W('如何') | (W('好用') + W('吗')) ),
       action = how_is_xxx),

  Rule(condition = W(pos='r') + W('是') + W(pos='nr') | \
                   W(pos='nr') + W('是') + W(pos='r') | \
                   (W('讲解') | W('介绍') | W('了解')) + Star(Any(), greedy=False) + W(pos='nr'),
       action = what_is_xxx),
]

In [7]:
for utt in ['宝马是什么',
            '我想了解一下宝马',
            '给我介绍一下宝马',
            '宝马这个牌子的汽车怎么样',
            '宝马如何呢',
            '宝马汽车好用吗',
            '宝马和奔驰比怎么样',
            '宝马和奔驰比哪个好',
            '宝马和奔驰比哪个更好',]:
  is_matched = False
  for rule in rules:
    db_query = rule.apply([Word(word, tag) for word, tag in pseg.cut(utt)])
    if db_query:
      for row in graph.query(db_query):
        print('Output:', uri2resource[row.o.toPython()])
        print()
      is_matched = True
      break
  if not is_matched:
    print('Not Matched:', [(word, tag) for word, tag in pseg.cut(utt)])

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.814 seconds.
Prefix dict has been built successfully.


Query: 宝马|nr  是|v  什么|r
Template: 'what is xxx?'
Output: 宝马（BMW）是德国豪华汽车品牌

Query: 了解|v  一下|m  宝马|nr
Template: 'what is xxx?'
Output: 宝马（BMW）是德国豪华汽车品牌

Query: 介绍|v  一下|m  宝马|nr
Template: 'what is xxx?'
Output: 宝马（BMW）是德国豪华汽车品牌

Query: 宝马|nr  这个|r  牌子|n  的|uj  汽车|n  怎么样|r
Template: 'how is xxx?'
Output: 德系大品牌值得信赖，各方面口碑都很好

Query: 宝马|nr  如何|r
Template: 'how is xxx?'
Output: 德系大品牌值得信赖，各方面口碑都很好

Query: 宝马|nr  汽车|n  好用|v  吗|y
Template: 'how is xxx?'
Output: 德系大品牌值得信赖，各方面口碑都很好

Query: 宝马|nr  和|c  奔驰|v  比|p  怎么样|r
Template: 'xxx compared to xxx?'
Output: 各有千秋，但是人生苦短，我选宝马

Query: 宝马|nr  和|c  奔驰|v  比|p  哪个|r  好|a
Template: 'xxx compared to xxx?'
Output: 各有千秋，但是人生苦短，我选宝马

Query: 宝马|nr  和|c  奔驰|v  比|p  哪个|r  更好|d
Template: 'xxx compared to xxx?'
Output: 各有千秋，但是人生苦短，我选宝马

